In [1]:
from typing_extensions import TypedDict, List, Optional, Union, Dict, Annotated, Literal
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, trim_messages
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
from langgraph.graph import END, StateGraph, START, MessagesState
from langchain_groq import ChatGroq
from langgraph.prebuilt import create_react_agent
# from team_tools import tavily_search_tool, arxiv_search_tool, web_scraper_tool, repl_tool
from qdrant_cloud_ops import initialize_selfquery_retriever, qdrant_vector_store
# from llm_chains import decomposition_chain, requires_decomposition, rephrase_chain, get_plan_chain, assign_chat_topic
from dotenv import load_dotenv
from pprint import pprint
import re
import functools
import operator
from langgraph.checkpoint.memory import MemorySaver

from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

from token_counter import tiktoken_counter




load_dotenv()

in_memory_store = InMemoryStore()
memory = MemorySaver()


llm = ChatGroq(model='llama-3.2-90b-vision-preview')


trimmer = trim_messages(
    max_tokens=5984,
    strategy="last",
    token_counter=tiktoken_counter,
    include_system=True,
    allow_partial=False,
)

qdrant_retriever = initialize_selfquery_retriever(llm, qdrant_vector_store=qdrant_vector_store)
qdrant_retriever_tool = qdrant_retriever.as_tool(
    name="retrieve_research_paper_texts",
    description="Search and return information from the vector database containing texts of several research papers, and scholarly articles",
)


Started Qdrant client.
Collection 'aireas-cloud' already exists.


/tmp/ipykernel_138179/1401692486.py:46: LangChainBetaWarning: This API is in beta and may change in the future.
  qdrant_retriever_tool = qdrant_retriever.as_tool(


In [ ]:
decomposer_chain = decomposition_chain(llm=llm)
check_query_chain = requires_decomposition(llm=llm)
rephraser_chain = rephrase_chain(llm=llm)
planner_chain = get_plan_chain(llm=llm)
assign_topic_chain = assign_chat_topic(llm=llm)

In [ ]:
assign_topic_chain.invoke(' ill be researching on animation generation using AI, partial attention networks and incorporating diffusion models for frame gen')

'AI-Generated Animation with Diffusion'

In [2]:
def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [3]:
k = create_react_agent(model=llm, tools=[qdrant_retriever_tool])


In [9]:
async for s in k.astream({'messages': [HumanMessage(content='hello, what is samurai.pdf main points?')]}, stream_mode='values'):
            if isinstance(s['messages'][-1], HumanMessage):
                continue
            else:
              print(s)

            # Check if the AI calls a tool
            # if 'tool_name' in msg:
            #     tool_name = msg['tool_name']
            #     query = msg.get('query', 'No query provided')

            #     print(f"\n[AI is calling tool: {tool_name}]")
            #     print(f"[Query sent to {tool_name}]: {query}")

            # # Check if the tool retrieved content
            # elif 'content' in msg:
            #     content = msg['content']
            #     print(f"\n[Tool Retrieved Content]: {content}")

            # # Final Answer Display
            # if 'final_answer' in msg:
            #     print(f"\n[Final Answer]: {msg['final_answer']}")


{'messages': [HumanMessage(content='hello, what is samurai.pdf main points?', additional_kwargs={}, response_metadata={}, id='09ebad3a-739b-44bc-ac67-9388bb378092'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8r5k', 'function': {'arguments': '{"__arg1": "samurai.pdf"}', 'name': 'retrieve_research_paper_texts'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 262, 'total_tokens': 284, 'completion_time': 0.111734198, 'prompt_time': 0.050164637, 'queue_time': 0.008238533999999999, 'total_time': 0.161898835}, 'model_name': 'llama-3.2-90b-vision-preview', 'system_fingerprint': 'fp_07b97e5459', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d2bc75ad-c6b6-41c6-b097-39b29f9ddebb-0', tool_calls=[{'name': 'retrieve_research_paper_texts', 'args': {'__arg1': 'samurai.pdf'}, 'id': 'call_8r5k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 262, 'output_tokens': 22, 'total_tokens': 284})]}
{'messages': [

BadRequestError: Error code: 400 - {'error': {'message': "'messages.6' : for 'role:tool' the following must be satisfied[('messages.6.content' : value must be a string)]", 'type': 'invalid_request_error'}}